<a href="https://colab.research.google.com/github/nxxk23/NAKBID/blob/main/NINE/web_scrap_drug_center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 🔹 ตั้งค่าข้อมูลล็อกอินและดาวน์โหลด
USERNAME = "chanatip0615@gmail.com"  # เปลี่ยนเป็นของคุณ
PASSWORD = "#chanatip2011"  # เปลี่ยนเป็นของคุณ
LOGIN_URL = "https://www.this.or.th/en/account/"
DOWNLOAD_PAGE = "https://www.this.or.th/en/download/"
SAVE_PATH = "C:/Users/Asus/Downloads/downloaded_file.zip"

# 🔹 ตั้งค่า Chrome
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 10)

try:
    # 🔹 ล็อกอิน
    driver.get(LOGIN_URL)
    wait.until(EC.presence_of_element_located((By.NAME, "log"))).send_keys(USERNAME)
    wait.until(EC.presence_of_element_located((By.NAME, "pwd"))).send_keys(PASSWORD)
    wait.until(EC.element_to_be_clickable((By.ID, "wp-submit"))).click()

    # 🔹 รอเปลี่ยนหน้า
    wait.until(EC.url_changes(LOGIN_URL))
    print("✅ ล็อกอินสำเร็จ")

    # 🔹 ไปยังหน้าดาวน์โหลด
    driver.get(DOWNLOAD_PAGE)

    # 🔹 รอปุ่มดาวน์โหลดปรากฏ
    try:
        download_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.t-button.t-download")))
        download_url = download_button.get_attribute("href")
        print(f"⏳ พบปุ่มดาวน์โหลด: {download_url}")
    except:
        print("❌ ไม่พบปุ่มดาวน์โหลด")
        driver.quit()
        exit()

    # 🔹 ดึง Cookie จาก Selenium
    cookies = {cookie['name']: cookie['value'] for cookie in driver.get_cookies()}
    print("✅ ดึง Cookie สำเร็จ")

finally:
    driver.quit()

# 🔹 ใช้ requests ดาวน์โหลดไฟล์พร้อม Cookie
print(f"⏳ กำลังดาวน์โหลดไฟล์จาก: {download_url}")
headers = {"User-Agent": "Mozilla/5.0"}  # ป้องกันเว็บบล็อกบอท
session = requests.Session()
session.cookies.update(cookies)

response = session.get(download_url, stream=True, headers=headers)

if response.status_code == 200:
    with open(SAVE_PATH, "wb") as file:
        for chunk in response.iter_content(1024):
            file.write(chunk)
    print(f"✅ ดาวน์โหลดไฟล์สำเร็จ: {SAVE_PATH}")
else:
    print(f"❌ ดาวน์โหลดไม่สำเร็จ (Error {response.status_code})")
